# Agent Demo with Simplified Connection Management

This notebook demonstrates an agent using multiple tool types with simplified connection management:
1. **Local Calculator**: Mathematical expressions via MCP
2. **Brave Search**: Web search via MCP (if API key available)
3. **Think Tool**: Internal reasoning tool

In [ ]:
import os
import sys
import nest_asyncio
nest_asyncio.apply()

parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)
from agents.agent import Agent, ModelConfig
from agents.tools.think import ThinkTool
from agents.tools.web_search import WebSearchServerTool
from agents.tools.code_execution import CodeExecutionServerTool

# 1. Configure tools and MCP servers

In [ ]:
# Standard Python tool
think_tool = ThinkTool()

# Python MCP server
calculator_server_path = os.path.abspath(os.path.join(os.getcwd(), "tools/calculator_mcp.py"))
calculator_server = {
    "type": "stdio",
    "command": "python",
    "args": [calculator_server_path]
}
print(f"Calculator server configured: {'Yes' if calculator_server else 'No'}")


# Brave MCP server written in TypeScript

brave_api_key = os.environ.get("BRAVE_API_KEY_BASE_DATA", "")
print(f"Brave API key available: {'Yes' if brave_api_key else 'No'}")
brave_search_server = {
    "type": "stdio",
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-brave-search"],
    "env": {
        "BRAVE_API_KEY": brave_api_key, 
        "PATH": f"{os.path.dirname('npx')}:" + os.environ.get("PATH", "")
    }
}
print(f"Brave search server configured: {'Yes' if brave_search_server else 'No'}")

# 2. Initialize agent with combined tools

In [ ]:
# Create agent config 
system_prompt = """
You are a helpful assistant with access to:
1. Web search (brave_web_search, brave_local_search)
2. Mathematical calculator (calculate) 
3. A tool to think and reason (think)

Always use the most appropriate tool for each task.
"""


# Initialize agent with standard tools and MCP servers
agent = Agent(
    name="Multi-Tool Agent",
    system=system_prompt,
    tools=[think_tool], 
    mcp_servers=[brave_search_server, calculator_server], 
    config=ModelConfig(
        model="claude-3-7-sonnet-20250219", 
        max_tokens=4096,
        temperature=1.0
    ),
    verbose=True
)

# 3. Run queries

In [ ]:
# Example query
agent.run("What's the square root of the OKC population in 2022")

In [ ]:
await agent.run_async("How many bananas will fit in an Toyota GR86?")

# 4. Example with Anthropic Server Tools

This example demonstrates using Anthropic's native server tools for web search and code execution.

In [ ]:
# Create Anthropic server tools
web_search_tool = WebSearchServerTool(
    name="web_search",
    max_uses=5,  # Limit to 5 searches per request
    blocked_domains=["example.com"]  # Example of blocking specific domains
)

code_execution_tool = CodeExecutionServerTool()

# Initialize agent with server tools
server_agent = Agent(
    name="Server Tools Agent",
    system="""
You are a helpful assistant with access to:
1. Web search for finding current information
2. Code execution for running Python code
3. Think tool for complex reasoning

Use these tools effectively to answer questions that require current data or calculations.
""",
    tools=[think_tool, web_search_tool, code_execution_tool],
    config=ModelConfig(
        model="claude-3-7-sonnet-20250219",
        max_tokens=4096,
        temperature=0.7
    ),
    verbose=True
)

In [ ]:
# Example 1: Use web search to find current information and code execution for analysis
server_agent.run("""
Search for the current population of Tokyo, Japan. 
Then write and execute Python code to calculate how many people that would be per square kilometer, 
given that Tokyo's area is approximately 2,194 square kilometers.
""")

In [ ]:
# Example 2: Combine web search with code execution for data analysis
await server_agent.run_async("""
Search for the current stock price of Apple (AAPL). 
Then use code execution to calculate what a $10,000 investment would be worth 
if the stock increases by 15% over the next year.
""")